In [1]:
import numpy as np
from numpy import newaxis
import tensorflow.compat.v1 as tf
import pandas as pd 
#from tensorflow import keras
#from keras import optimizers
from tensorflow.keras.optimizers import schedules
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import TimeDistributed,Input
from tensorflow.keras.layers import LSTM, GRU
from tensorflow.keras import Sequential, Model
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import Callback,ReduceLROnPlateau,EarlyStopping,LambdaCallback,TensorBoard
from tensorflow.keras import metrics

from tensorflow.keras.callbacks import History 
from tensorflow.keras import backend as K

from scipy.sparse import csc_matrix
from scipy.sparse.linalg import lsqr
from numpy import linalg as LA


import random 
from scipy.constants import Planck
from scipy.optimize import least_squares,minimize
import datetime



import itertools
from itertools import product
import os
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from qutip import *
import matplotlib.pyplot as plt
%matplotlib
plt.style.use('fivethirtyeight')

from Functions import *
from Classes import *




Using matplotlib backend: Qt5Agg


In [2]:
n=1
d=np.power(2,n)
os.chdir('/home/andrius/Desktop/RNN Estados ML Jeff/Dados')
theta=np.loadtxt('angulos.txt')

txt = open('(0)(0)ML.txt').read()
txt = [item.split() for item in txt.split('\n\n')]
estado00=text_to_rho(txt,d)
estado00_qutip=text_to_rho_qutip(text_to_rho(txt,d),d)
#Aplicando produto tensorial com estado |0> 
#conforme experimento do jeferson
#for i in range(len(estado00_qutip)):
#    estado00_qutip[i]=tensor(estado00_qutip[i],zero)

txt = open('(1)(0)ML.txt').read()
txt = [item.split() for item in txt.split('\n\n')]
estado_10=text_to_rho(txt,d)
estado_10_qutip=text_to_rho_qutip(text_to_rho(txt,d),d)
#for i in range(len(estado00_qutip)):
#    estado_10_qutip[i]=tensor(estado_10_qutip[i],zero)

txt = open('estado1ML.txt').read()
txt = [item.split() for item in txt.split('\n\n')]
estado1=text_to_rho(txt,d)
estado1_qutip=text_to_rho_qutip(text_to_rho(txt,d),d)
#for i in range(len(estado1_qutip)):
#    estado1_qutip[i]=tensor(estado1_qutip[i],zero)


txt = open('estado2ML.txt').read()
txt = [item.split() for item in txt.split('\n\n')]
estado2=text_to_rho(txt,d)
estado2_qutip=text_to_rho_qutip(text_to_rho(txt,d),d)
#for i in range(len(estado1_qutip)):
#    estado2_qutip[i]=tensor(estado2_qutip[i],zero)


txt = open('estado3ML.txt').read()
txt = [item.split() for item in txt.split('\n\n')]
estado3=text_to_rho(txt,d)
estado3_qutip=text_to_rho_qutip(text_to_rho(txt,d),d)
#for i in range(len(estado3_qutip)):
#    estado3_qutip[i]=tensor(estado3_qutip[i],zero)


txt = open('estado4ML.txt').read()
txt = [item.split() for item in txt.split('\n\n')]
estado4=text_to_rho(txt,d)
estado4_qutip=text_to_rho_qutip(text_to_rho(txt,d),d)
#for i in range(len(estado4_qutip)):
#   estado4_qutip[i]=tensor(estado4_qutip[i],zero)


txt = open('estado5ML.txt').read()
txt = [item.split() for item in txt.split('\n\n')]
estado5=text_to_rho(txt,d)
estado5_qutip=text_to_rho_qutip(text_to_rho(txt,d),d)
#for i in range(len(estado5_qutip)):
#    estado5_qutip[i]=tensor(estado5_qutip[i],zero)


txt = open('estado6ML.txt').read()
txt = [item.split() for item in txt.split('\n\n')]
estado6=text_to_rho(txt,d)
estado6_qutip=text_to_rho_qutip(text_to_rho(txt,d),d)
#for i in range(len(estado6_qutip)):
#    estado6_qutip[i]=tensor(estado6_qutip[i],zero)


lista_estados=[estado1,estado2,estado3,estado4,estado5,estado6,estado00,estado_10]
lista_estados_qutip=[estado1_qutip,estado2_qutip,estado3_qutip,estado4_qutip,estado5_qutip,estado6_qutip,estado00_qutip,estado_10_qutip]
#lista_estados_random=random.sample(lista_estados,len(lista_estados))
estados=np.array(np.stack((lista_estados),axis=0))


verificarHermit(lista_estados_qutip)

560

Para escolher como dividir os dados de treino, temos as seguintes opçoes:
1- Como eu sempre fiz: Dividir x estados de treino e y de teste, com x+y= numero de estados medidos
    i-fornece val_loss referente a tipo de estados nao vistos pela rede
2- Usar todos estados e separar para o teste os dados temporais finais de cada estado
    i-fornece val_loss referente a steps nao vistos


In [18]:
train_test_split=2
n_steps=17

A_op=np.array(qeye(d))
divid=np.int((d**2))
data_dim=divid*2

rho=estados[:-train_test_split,:,:]
rho_test=estados[-train_test_split:,:,:]

#Mudar estados selecionados para teste para ver como val loss se comporta
#rho_test,rho=np.split(estados,[3],axis=0)

x_train,y_train,x_test,y_test=dataSplit(estados,train_test_split,n_steps)
data=np.stack([x_train,y_train],axis=-1)
data_test=np.stack([x_test,y_test],axis=-1)

In [14]:
x_train.shape

(312, 17, 8)

In [4]:
x_test.shape

(104, 17, 8)

In [6]:
#Config

pesos=[]
neurons=20
delta_t=theta[1] # delta_t constante,nao precisa passara para o data gen !
rho_input_tensor=tf.convert_to_tensor(x_train)
delta_t_tensor=tf.dtypes.cast(tf.convert_to_tensor(delta_t),tf.complex128)


model=tf.keras.Sequential()
model.add(tf.keras.Input(shape=(n_steps,data_dim)))
model.add(tf.keras.layers.GRU(neurons,return_sequences=True ,kernel_initializer='random_normal'))
model.add(tf.keras.layers.GRU(neurons,return_sequences=True ,kernel_initializer='random_normal'))
#model.add(tf.keras.layers.Dropout(.2))
model.add(tf.keras.layers.Dense(data_dim))

#generator=DataGenerator(x=x_train,y=y_train,batch_size=batch_size)
adam = tf.keras.optimizers.RMSprop(lr=0.001,rho=0.9) 
#Callbacks
early_stop =EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=0, 
                                                     mode='min', baseline=None, restore_best_weights=True)

print_weights = LambdaCallback(on_epoch_end=lambda batch, logs: pesos.append(model.layers[1].get_weights()))

tensorboard = TensorBoard(log_dir="logs/{}".format(1),histogram_freq=1)

model.compile(loss=Z(delta_t_tensor,d),optimizer=adam)


#history=model.fit_generator(generator=DataGenerator(batch_size,x_train,y_train,x_train_0),verbose=True,
#                            validation_data=(DataGenerator(batch_size,x_test,y_test,x_test_0)),
#                            epochs=100) #callbacks=[early_stop]callbacks=[print_weights,History]
history=model.fit(x=x_train,y=data,batch_size=17,validation_data=(x_test,data_test),epochs=50)

Epoch 1/50
(None, None, 2, 2)
(None, None, 2, 2)
19/19 [==============================] - 7s 82ms/step - loss: 0.0425 - val_loss: 0.0271
Epoch 2/50
19/19 [==============================] - 1s 41ms/step - loss: 0.0419 - val_loss: 0.0268
Epoch 3/50
19/19 [==============================] - 1s 42ms/step - loss: 0.0417 - val_loss: 0.0271
Epoch 4/50
19/19 [==============================] - 1s 41ms/step - loss: 0.0416 - val_loss: 0.0269
Epoch 5/50
19/19 [==============================] - 1s 43ms/step - loss: 0.0414 - val_loss: 0.0267
Epoch 6/50
19/19 [==============================] - 1s 43ms/step - loss: 0.0413 - val_loss: 0.0266
Epoch 7/50
19/19 [==============================] - 1s 53ms/step - loss: 0.0411 - val_loss: 0.0266
Epoch 8/50
19/19 [==============================] - 1s 59ms/step - loss: 0.0407 - val_loss: 0.0274
Epoch 9/50
19/19 [==============================] - 1s 59ms/step - loss: 0.0407 - val_loss: 0.0264
Epoch 10/50
19/19 [==============================] - 1s 55ms/step - los

test_size=3 n split=3 loss: 0.0414 - val_loss: 0.0339

test_size=1 n split=3 loss: 0.0367 - val_loss: 0.0349

test_size=1  n split= 1 loss: 0.0388 - val_loss: 0.0263

In [7]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [7]:
print(history.history.keys())

dict_keys(['loss', 'val_loss'])


In [19]:
#Config
timesteps=len(estados[0])
step_index=len(estados[0])-1

divid=np.int(data_dim/2)  
H=Qobj(np.zeros((d,d)))
timestamps=np.linspace(0,5.6,timesteps)
kwargs={'estados_getgama':rho,'estados_test':estados,'timestamps': timestamps,'model':model,'timestep':timesteps, 'step_index':step_index,'divid':divid}  
pred=ModeloPredicao(**kwargs)
fid_df,norm_df,prediçao_qutip=pred.predicao()
gama_mediatotal=pred.gama_mediatotal


In [20]:
fid_df

,Estado0,Estado1,Estado2,Estado3,Estado4,Estado5,Estado6,Estado7
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.956433,0.999111,0.999753,0.998508,0.999428,0.998891,0.997241,0.996342
2,0.969176,0.999034,0.999650,0.999113,0.999308,0.999391,0.999891,0.998550
3,0.954891,0.998641,0.999416,0.999206,0.997950,0.998584,0.999901,0.997840
4,0.990080,0.998191,0.999261,0.998941,0.998394,0.998918,0.999843,0.994362
...,...,...,...,...,...,...,...,...
65,0.899969,0.899012,0.904899,0.879527,0.840193,0.897275,0.998250,0.736801
66,0.886069,0.906526,0.891590,0.884081,0.836915,0.884958,0.998301,0.736434
67,0.877883,0.885357,0.894704,0.859393,0.822372,0.888811,0.998760,0.735129
68,0.881324,0.886717,0.884377,0.864971,0.819100,0.872994,0.998960,0.733972


In [10]:
gama_mediatotal

array([-0.06603538-0.07337612j, -0.01283073-0.00842983j,
       -0.0063292 +0.01088832j,  0.02613212+0.05505827j])

In [11]:
os.mkdir("/home/andrius/Desktop/RNN Estados ML Jeff/Resultados/+"+str(datetime.datetime.now().strftime("%Y-%m-%d %H-%M")))


In [12]:
os.chdir("/home/andrius/Desktop/RNN Estados ML Jeff/Resultados/+"+str(datetime.datetime.now().strftime("%Y-%m-%d %H-%M")))


np.savetxt('./fidelidades.csv', fid_df)
np.savetxt('./tracedist.csv', norm_df)
np.savetxt('./Operadores.csv', gama_mediatotal)


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig('learning.png')

#file=open('./resultados'+str(datetime.datetime.now().strftime("%Y-%m-%d %H-%M-%S"))+'.txt', 'w')
file=open("/home/andrius/Desktop/RNN Estados ML Jeff/Resultados/+overfitting/resultados.txt",'w')
file.write("Coeficientes médios encontrados={0}\n".format(gama_mediatotal)) 
file.write("Estados tomografados para teste={0}\n".format(train_test_split))
file.write("Steps temporais={0}\n".format(n_steps))    
file.write("Timesteps de validação={0}\n".format(timestamps))

file.close()


# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")

NameError: name 'train' is not defined